In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.stats.diagnostic as dg
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy.stats import norm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sns.mpl.rcParams['figure.figsize'] = (15.0, 6.5)
pd.set_option('display.max_rows', 1000) # set max rows to show in any dataframe

# Data preparation

## Get our main Dataset - RIO

In [ ]:
datasets_folder = '.'

df_main = pd.read_csv(datasets_folder + '/RIO_VA_FOR_KIS-data.csv')
# looks like there are some variables whose purpose is only to describe other variables already in the data frame, so we won't consider them further down the analysis
df_main_edit = df_main.drop(['TIME_FORMAT', 'FREQ_DESC', 'TIME_FORMAT_DESC',
       'TIME_PERIOD_DESC', 'NACE_R2_DESC', 'NA_ITEM_DESC', 'UNIT_DESC',
       'GEO_DESC'], axis=1)
# FREQ and NA_ITEM only have one type of observation, respectively: 'A' (meaning annual data) and 'B1G' (meaning gross value-added); we can discard them as well then, since they don't add any more information to what we have already:
df_main_edit = df_main_edit.drop(['FREQ', 'NA_ITEM'], axis=1)
# 'TIME_PERIOD' as datetime type:
df_main_edit['TIME_PERIOD'] = pd.to_datetime(df_main_edit['TIME_PERIOD'], format='%Y') 

# 'NACE_R2', 'GEO', 'UNIT' as category type:
for col in ['NACE_R2', 'GEO', 'UNIT']:
    df_main_edit[col] = df_main_edit[col].astype('category')
# Eliminating the uneeded observations:
df_main_clean = df_main_edit.drop(df_main_edit[df_main_edit['GEO'] == 'EU28'].index)
df_main_clean = df_main_clean.drop(df_main_clean[df_main_clean['NACE_R2'] == 'TOTAL_INDICATOR'].index)
# Let's eliminate all the observations whose UNIT is CP_MNAC (Current prices, million units of national currency):
df_main_clean = df_main_clean.drop(df_main_clean[df_main_clean['UNIT'] == 'CP_MNAC'].index)
df_main_pivoted = df_main_clean.pivot(index=['TIME_PERIOD', 'GEO'], columns=['NACE_R2','UNIT'], values='OBS_VALUE')
# If we want access to 'TIME_PERIOD' and 'GEO' as before, we must reset the index; otherwise we won't be able to access it's contents
df_main_pivoted = df_main_pivoted.reset_index()

# Because the common years we have across the data from all datasets ranges from 2008 to 2015, we create a new df_main containing only information regarding these years:
df_main_cdated = df_main_pivoted[(df_main_pivoted['TIME_PERIOD'].dt.year >= 2008) & (df_main_pivoted['TIME_PERIOD'].dt.year <= 2015)]

df_main_cdated.head()


## Get the Venture dataset

In [ ]:
# Read auxiliary dataset: 'Venture capital investments'
df_venture = pd.read_csv(datasets_folder + '/RIO_VENTURE-data.csv')
# as what happens with our main dataset, in this one there are also many variables that we can discard before we analyse the data any further:
df_venture_edit = df_venture.drop(['FREQ', 'TIME_FORMAT', 'EXPEND', 'UNIT_DESC', 'OBS_STATUS', 'FREQ_DESC', 'TIME_FORMAT_DESC',
       'TIME_PERIOD_DESC', 'OBS_STATUS_DESC', 'EXPEND_DESC', 'GEO_DESC'], axis=1)
# Let's transform 'TIME_PERIOD' as datetime type:
df_venture_edit['TIME_PERIOD'] = pd.to_datetime(df_venture_edit['TIME_PERIOD'], format='%Y') 
df_venture_pivoted = df_venture_edit.pivot(index=['TIME_PERIOD', 'GEO'], columns=['UNIT'], values='OBS_VALUE')
# If we want access to 'TIME_PERIOD' and 'GEO' as before, we must reset the index; otherwise we won't be able to access it's contents
df_venture_pivoted = df_venture_pivoted.reset_index()
df_venture_cdated = df_venture_pivoted[(df_venture_pivoted['TIME_PERIOD'].dt.year >= 2008) & (df_venture_pivoted['TIME_PERIOD'].dt.year <= 2015)]
df_venture_cdated.head()


This data set is the one that has the least countries represented, so we have to match the data on the others based on this one. It means our data will be on these 20 countries.

In [ ]:
df_venture_cdated.GEO.unique().size

In [ ]:
countries_main = df_venture_cdated.GEO.unique()
countries_main

Let's change the main dataset in order to keep just these countries:

In [ ]:
df_main_cdated = df_main_cdated[df_main_cdated['GEO'].isin(countries_main)]


## Now join Venture capital Dataset to main dataset:

In [ ]:
joint_dataset = df_main_cdated.merge(df_venture_cdated, how="left", left_on=["TIME_PERIOD", "GEO"], right_on=["TIME_PERIOD", "GEO"])
joint_dataset = joint_dataset.drop(columns=joint_dataset.columns[[2, 3]])
col_rename = {}
# Rename new columns
for col in [11, 12, 13]:
    col_rename[joint_dataset.columns[col]] = "VENTURE_" + joint_dataset.columns[col]
# Rename leveled columns
for col in range(2,11):
    col_rename[joint_dataset.columns[col]] = joint_dataset.columns[col][0] + "_" + joint_dataset.columns[col][1]

joint_dataset = joint_dataset.rename(columns=col_rename)
joint_dataset.head()

## Get the Employment dataset

In [ ]:
df_tsc00011 = pd.read_csv(datasets_folder + "/tsc00011.csv")
df_tsc00011_1= df_tsc00011.drop(["unit", "sex"], axis="columns")
# refactor dataset
df_tsc00011_melted = df_tsc00011_1.melt(["nace_r2", "geo\\time"], var_name='year', value_name='value')
# clean up value field
df_tsc00011_melted["value"] = df_tsc00011_melted["value"].replace(to_replace="[\sbdep]", value="", regex=True)
df_tsc00011_melted["value"] = df_tsc00011_melted["value"].replace(to_replace="[,]", value=".", regex=True)
# set year column to datetime
df_tsc00011_melted['year'] = pd.to_datetime(df_tsc00011_melted['year'], format='%Y') 
# set value column to numeric
df_tsc00011_melted["value"] = pd.to_numeric(df_tsc00011_melted["value"], errors="coerce")
# filter for the main country list only
countries_tsc00011 = df_tsc00011_melted.loc[df_tsc00011_melted['geo\\time'].isin(countries_main)]
# Refactor to be able to merge to main dataset
tsc00011_ready = countries_tsc00011.pivot_table(index=["year", "geo\\time"], columns="nace_r2", values="value").reset_index()
tsc00011_ready.head()

## Now also join employment dataset to main dataset.

In [ ]:
joint_dataset = joint_dataset.merge(tsc00011_ready, how="left", left_on=["TIME_PERIOD", "GEO"], right_on=["year", "geo\\time"])
joint_dataset = joint_dataset.drop(columns=["year", "geo\\time"])
col_rename = {"C_HTC_MH":"EMPLOYMENT_C_HTC_MH", "KIS": "EMPLOYMENT_KIS"}
joint_dataset = joint_dataset.rename(columns=col_rename)

joint_dataset.head()

## Now get the GERDS dataset

In [ ]:
gerds = pd.read_csv(datasets_folder + "/ESTAT_rd_e_gerdtot-data.csv")
# Only pick the necessary values - per sector and interesting units
gerds_edit = gerds[gerds.UNIT.isin(["MIO_EUR", "EUR_HAB", "MIO_PPS_KP05","PC_GDP"]) & gerds.SECTPERF.isin(["BES", "GOV", "HES", "PNP"])] \
    .drop(["FREQ", "TIME_FORMAT", "OBS_STATUS", "FREQ_DESC", "TIME_FORMAT_DESC", "TIME_PERIOD_DESC", "OBS_STATUS_DESC", "SECTPERF_DESC", "UNIT_DESC", "GEO_DESC"], axis="columns")
# Only keep countries form Main dataset
gerds_edit=gerds_edit[gerds_edit["GEO"].isin(countries_main)]
# Convert year into datetime
# Let's transform 'TIME_PERIOD' as datetime type:
gerds_edit['TIME_PERIOD'] = pd.to_datetime(gerds_edit['TIME_PERIOD'], format='%Y')
# Only use years 2008 to 2015
gerds_edit = gerds_edit[(gerds_edit['TIME_PERIOD'].dt.year >= 2008) & (gerds_edit['TIME_PERIOD'].dt.year <= 2015)]
# Refactor to be joinable
gerds_joinable = gerds_edit.pivot_table(index=["TIME_PERIOD", "GEO"], columns=["SECTPERF", "UNIT"], values="OBS_VALUE").reset_index()
gerds_joinable.head()

# display(gerds_edit.info())
# display(gerds_edit.groupby(["SECTPERF", "UNIT"]).describe())
# display("missing values per column:", gerds_edit.isna().sum())
# display(gerds_edit[gerds_edit.OBS_VALUE.isna()].groupby(["GEO", "SECTPERF", "UNIT", "TIME_PERIOD"]).count())

## Now join GERDS to main dataset

In [ ]:
joint_dataset2 = joint_dataset.merge(gerds_joinable, how="left", left_on=["TIME_PERIOD", "GEO"], right_on=["TIME_PERIOD", "GEO"])
col_rename = {}
# Rename leveled columns
for col in joint_dataset2.columns[-16:]:
    col_rename[col] = "GERDS_" + col[0] + "_" + col[1]
col_rename
joint_dataset2 = joint_dataset2.rename(columns=col_rename)
joint_dataset2.head()

## Recheck some basic descriptive statistics, now that we have the final dataset we'll work with

In [ ]:
joint_dataset2.shape

In [ ]:
joint_dataset2.info()

In [ ]:
joint_dataset2.describe()

## Missing Values Analysis and Processing

In [ ]:
# Check how many missing values we have for each variable:
print("Total of missing values:")
display(joint_dataset2.isnull().sum().sum())
print("Number of missing values in each variable:")
joint_dataset2.isnull().sum()

In [ ]:
# In which years and respective country we have missing values in variable C_HTC_PERC_OF_TOTAL:
display(joint_dataset2[joint_dataset2['C_HTC_PERC_OF_TOTAL'].isnull()].groupby(['TIME_PERIOD','GEO']).size())

In [ ]:
# Missing Values for C_HTC_PERC_OF_TOTAL:
# Luxembourg, from 2008 to 2015
# Ireland, 2015

In [ ]:
# In which years and respective country we have missing values in variable C_HTC_PERC_OF_MANUF:
display(joint_dataset2[joint_dataset2['C_HTC_PERC_OF_MANUF'].isnull()].groupby(['TIME_PERIOD','GEO']).size())

In [ ]:
# Missing Values for C_HTC_PERC_OF_TOTAL:
# Luxembourg, from 2008 to 2015
# Ireland, 2015

In [ ]:
# In which years and respective country we have missing values in variable C_HTC_CP_MEUR:
display(joint_dataset2[joint_dataset2['C_HTC_CP_MEUR'].isnull()].groupby(['TIME_PERIOD','GEO']).size())

In [ ]:
# Missing Values for C_HTC_PERC_OF_TOTAL:
# Luxembourg, from 2008 to 2015
# Ireland, 2015

In [ ]:
# In which years and respective country we have missing values in variable GERDS_PNP_EUR_HAB:
display(joint_dataset2[joint_dataset2['GERDS_PNP_EUR_HAB'].isnull()].groupby(['GEO','TIME_PERIOD']).size())

In [ ]:
# Missing Values for GERDS_PNP_EUR_HAB:
# 2008: DE, HU, IE, LU, NL
# 2009: DE, HU, IE, NL
# 2010: DE, HU, IE, NL
# 2011: DE, HU, IE, NL
# 2012: DE, HR, HU, IE, NL
# 2013: DE, HR, HU, IE, LU, NL
# 2014: DE, HR, HU, IE, LU, NL
# 2015: DE, HR, HU, IE, LU

In [ ]:
# In which years and respective country we have missing values in variable GERDS_PNP_MIO_EUR:
display(joint_dataset2[joint_dataset2['GERDS_PNP_MIO_EUR'].isnull()].groupby(['GEO', 'TIME_PERIOD']).size())

In [ ]:
# Missing Values for GERDS_PNP_MIO_EUR:
# 2008: DE, HU, IE, LU, NL
# 2009: DE, HU, IE, NL
# 2010: DE, HU, IE, NL
# 2011: DE, HU, IE, NL
# 2012: DE, HR, HU, IE, NL
# 2013: DE, HR, HU, IE, LU, NL
# 2014: DE, HR, HU, IE, LU, NL
# 2015: DE, HR, HU, IE, LU

In [ ]:
# In which years and respective country we have missing values in variable GERDS_PNP_MIO_PPS_KP05:
display(joint_dataset2[joint_dataset2['GERDS_PNP_MIO_PPS_KP05'].isnull()].groupby(['GEO', 'TIME_PERIOD']).size())

In [ ]:
# Missing Values for GERDS_PNP_MIO_PPS_KP05:
# 2008: DE, HU, IE, LU, NL
# 2009: DE, HU, IE, NL
# 2010: DE, HU, IE, NL
# 2011: DE, HU, IE, NL
# 2012: DE, HR, HU, IE, NL
# 2013: DE, HR, HU, IE, LU, NL
# 2014: DE, HR, HU, IE, LU, NL
# 2015: DE, HR, HU, IE, LU

In [ ]:
# In which years and respective country we have missing values in variable GERDS_PNP_PC_GDP:
display(joint_dataset2[joint_dataset2['GERDS_PNP_PC_GDP'].isnull()].groupby(['GEO','TIME_PERIOD']).size())

In [ ]:
# Missing Values for GERDS_PNP_PC_GDP:
# 2008: DE, HU, IE, LU, NL
# 2009: DE, HU, IE, NL
# 2010: DE, HU, IE, NL
# 2011: DE, HU, IE, NL
# 2012: DE, HR, HU, IE, NL
# 2013: DE, HR, HU, IE, LU, NL
# 2014: DE, HR, HU, IE, LU, NL
# 2015: DE, HR, HU, IE, LU

### Processing Missing Values

#### There are two main groups of missing values in the data we have so far:  
    a) those related to the value added in High-Tech Manufacturing sectors in Luxembourg and Ireland, in some years;  
    b) those related to expenditure data in R&D in the Private Non-Profit Sector in Germany (DE), Croatia (HR), Hungary (HU), Ireland (IE), Luxembourg (LU), and Netherlands (NL) in some years  
  
#### What will we do about these missing values?  
    - In the data about a), with some research we come to the conclusion that Luxembourg indeed has no High-Tech Manufacturing, so we will input all the related values with 0; about Ireland, the information we can get suggests that this country has High-Tech Manufacturing - so, based on this, we decided to input the previous year's values to the year that is missing (ie: high tech data from 2015 == high tech data from 2014)  
    - In the data about b) we can assume that the missing values represent the absent of expenditure in Non Profit Private Sector in R&D for those countries/years; in this case too, we will input the missing values with 0

#### Inputing the missing values with data accordingly:

In [ ]:
#High Tech in Luxembourg:
rows = (joint_dataset2['GEO'] == 'LU')
cols = ['C_HTC_PERC_OF_TOTAL', 'C_HTC_PERC_OF_MANUF', 'C_HTC_CP_MEUR']

joint_dataset2.loc[rows, cols] = joint_dataset2.loc[rows, cols].fillna(value=0)

#High Tech in Ireland:
rows = (joint_dataset2['GEO'] == 'IE')
cols = ['C_HTC_PERC_OF_TOTAL', 'C_HTC_PERC_OF_MANUF', 'C_HTC_CP_MEUR']
ie_previous_year = {'C_HTC_PERC_OF_TOTAL': joint_dataset2.loc[(joint_dataset2['GEO'] == 'IE') & (joint_dataset2['TIME_PERIOD'] == "2014"), ['C_HTC_PERC_OF_TOTAL']].iloc[0,0], 'C_HTC_PERC_OF_MANUF': joint_dataset2.loc[(joint_dataset2['GEO'] == 'IE') & (joint_dataset2['TIME_PERIOD'] == "2014"), ['C_HTC_PERC_OF_MANUF']].iloc[0,0],'C_HTC_CP_MEUR': joint_dataset2.loc[(joint_dataset2['GEO'] == 'IE') & (joint_dataset2['TIME_PERIOD'] == "2014"), ['C_HTC_CP_MEUR']].iloc[0,0]}

joint_dataset2.loc[rows, cols] = joint_dataset2.loc[rows, cols].fillna(value=ie_previous_year)

In [ ]:
joint_dataset2.isnull().sum()

In [ ]:
# Values in Expenditure with R&D in Non-Profit Private Organizations

# because all countries/year shall have the same processing, it is pretty straightforward:
cols = ['GERDS_PNP_EUR_HAB', 'GERDS_PNP_MIO_EUR', 'GERDS_PNP_MIO_PPS_KP05', 'GERDS_PNP_PC_GDP']

joint_dataset2.loc[:, cols] = joint_dataset2.loc[:, cols].fillna(value=0)

In [ ]:
joint_dataset2.isnull().sum()

## Distribution analysis, skewness and kurtosis

In [ ]:
joint_dataset2.kurtosis() # kurtosis for each metric variable

In [ ]:
joint_dataset2.skew() # skewness for each metric variable

In [ ]:
# filter distributions that have values for skewness out of the Normal range:
not_symmetric = joint_dataset2.skew()[(joint_dataset2.skew() > 1) | (joint_dataset2.skew() < -1)]
display(not_symmetric)

# filter distributions that have values for kurtosis out of the Normal range:
peaked_or_flat = joint_dataset2.kurtosis()[(joint_dataset2.kurtosis() > 1) | (joint_dataset2.kurtosis() < -1)]
display(peaked_or_flat)

In [ ]:
# plot every variable distribution

for elem in joint_dataset2.columns[2:]:
    sns.displot(joint_dataset2[elem], kind="kde")

In [ ]:
# filter the variables whose values for both skewness and kurtosis indicate a Normal-like distribution and plot them:
normal_skew = joint_dataset2.skew()[(joint_dataset2.skew() < 1) & (joint_dataset2.skew() > -1)]
normal_kurt = joint_dataset2.kurtosis()[(joint_dataset2.kurtosis() < 1) & (joint_dataset2.kurtosis() > -1)]
print('Variables that have Normal Distribution:\n')
normal = []
for elem in normal_skew.index:
    if elem in normal_kurt.index:
        normal.append(elem)
        sns.displot(joint_dataset2[elem], kind="kde")

# Correlation analysis - the road to feature selection

## Correlations between metric variables

In [ ]:
joint_dataset2.corr() # Computes pairwise correlation of columns, excluding NA/null values

In [ ]:
# What are the highest correlations?
for col in joint_dataset2.corr().columns:
    display(joint_dataset2.corr()[col][((joint_dataset2.corr()[col] > 0.5) | (joint_dataset2.corr()[col] < -0.5))])

In [ ]:
# Let's get a visual aid of the values above:
corr_matrix = joint_dataset2.corr()
high_corr = corr_matrix[(corr_matrix>=0.5) | (corr_matrix<=-0.5)]
plt.figure(figsize=(30,13))
sns.heatmap(high_corr, annot=True, cmap="Reds")

## Based on these correlations, there are some variables that we will discard

In [ ]:
# Clear unneeded columns
joint_dataset2 = joint_dataset2.drop(["GERDS_BES_MIO_PPS_KP05", "GERDS_GOV_MIO_PPS_KP05", "GERDS_HES_MIO_PPS_KP05", "GERDS_PNP_MIO_PPS_KP05", "GERDS_BES_PC_GDP", "GERDS_GOV_PC_GDP", "GERDS_HES_PC_GDP", "GERDS_PNP_PC_GDP"], axis="columns")
joint_dataset2.shape

In [ ]:
# Let's get a visual aid of the values above:
corr_matrix = joint_dataset2.corr()
high_corr = corr_matrix[(corr_matrix>=0.75) | (corr_matrix<=-0.75)]
plt.figure(figsize=(30,13))
sns.heatmap(high_corr, annot=True, cmap="Greens")

# Outliers Analysis

In [ ]:
from scipy import stats
# discriminate outliers in each variable; we use the threshold of 3 standard deviations away from the mean as the criteria to identify the outliers:
for col in joint_dataset2.iloc[:,2:].columns:
    display(joint_dataset2[["TIME_PERIOD", "GEO", col]][(np.abs(stats.zscore(joint_dataset2[col])) > 3)])

In [ ]:
# Scatter plot - C_HTC_M_CP_MEUR

d =  joint_dataset2
x1 = joint_dataset2["C_HTC_M_CP_MEUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of C_HTC_M_CP_MEUR
q1 = joint_dataset2['C_HTC_M_CP_MEUR'].quantile(.25)
q3 = joint_dataset2['C_HTC_M_CP_MEUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - KIS_PERC_OF_SERV

d =  joint_dataset2
x1 = joint_dataset2["KIS_PERC_OF_SERV"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of KIS_PERC_OF_SERV
q1 = joint_dataset2['KIS_PERC_OF_SERV'].quantile(.25)
q3 = joint_dataset2['KIS_PERC_OF_SERV'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - VENTURE_MIO_EUR

d =  joint_dataset2
x1 = joint_dataset2["VENTURE_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of VENTURE_MIO_EUR
q1 = joint_dataset2['VENTURE_MIO_EUR'].quantile(.25)
q3 = joint_dataset2['VENTURE_MIO_EUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - VENTURE_NR_COMP 

d =  joint_dataset2
x1 = joint_dataset2["VENTURE_NR_COMP"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of VENTURE_NR_COMP
q1 = joint_dataset2['VENTURE_NR_COMP'].quantile(.25)
q3 = joint_dataset2['VENTURE_NR_COMP'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - VENTURE_PC_GDP

d =  joint_dataset2
x1 = joint_dataset2["VENTURE_PC_GDP"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of VENTURE_PC_GDP
q1 = joint_dataset2['VENTURE_PC_GDP'].quantile(.25)
q3 = joint_dataset2['VENTURE_PC_GDP'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_BES_MIO_EUR

d =  joint_dataset2
x1 = joint_dataset2["GERDS_BES_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_BES_MIO_EUR
q1 = joint_dataset2['GERDS_BES_MIO_EUR'].quantile(.25)
q3 = joint_dataset2['GERDS_BES_MIO_EUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_GOV_EUR_HAB

d =  joint_dataset2
x1 = joint_dataset2["GERDS_GOV_EUR_HAB"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_GOV_EUR_HAB
q1 = joint_dataset2['GERDS_GOV_EUR_HAB'].quantile(.25)
q3 = joint_dataset2['GERDS_GOV_EUR_HAB'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_GOV_MIO_EUR

d =  joint_dataset2
x1 = joint_dataset2["GERDS_GOV_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_GOV_MIO_EUR
q1 = joint_dataset2['GERDS_GOV_MIO_EUR'].quantile(.25)
q3 = joint_dataset2['GERDS_GOV_MIO_EUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_HES_EUR_HAB 

d =  joint_dataset2
x1 = joint_dataset2["GERDS_HES_EUR_HAB"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_HES_EUR_HAB
q1 = joint_dataset2['GERDS_HES_EUR_HAB'].quantile(.25)
q3 = joint_dataset2['GERDS_HES_EUR_HAB'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_HES_MIO_EUR 

d =  joint_dataset2
x1 = joint_dataset2["GERDS_HES_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Scatter plot - C_HTC_M_CP_MEUR

d =  joint_dataset2
x1 = joint_dataset2["C_HTC_M_CP_MEUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of C_HTC_M_CP_MEUR
q1 = joint_dataset2['C_HTC_M_CP_MEUR'].quantile(.25)
q3 = joint_dataset2['C_HTC_M_CP_MEUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - KIS_PERC_OF_SERV

d =  joint_dataset2
x1 = joint_dataset2["KIS_PERC_OF_SERV"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of KIS_PERC_OF_SERV
q1 = joint_dataset2['KIS_PERC_OF_SERV'].quantile(.25)
q3 = joint_dataset2['KIS_PERC_OF_SERV'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - VENTURE_MIO_EUR

d =  joint_dataset2
x1 = joint_dataset2["VENTURE_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of VENTURE_MIO_EUR
q1 = joint_dataset2['VENTURE_MIO_EUR'].quantile(.25)
q3 = joint_dataset2['VENTURE_MIO_EUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - VENTURE_NR_COMP 

d =  joint_dataset2
x1 = joint_dataset2["VENTURE_NR_COMP"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of VENTURE_NR_COMP
q1 = joint_dataset2['VENTURE_NR_COMP'].quantile(.25)
q3 = joint_dataset2['VENTURE_NR_COMP'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - VENTURE_PC_GDP

d =  joint_dataset2
x1 = joint_dataset2["VENTURE_PC_GDP"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of VENTURE_PC_GDP
q1 = joint_dataset2['VENTURE_PC_GDP'].quantile(.25)
q3 = joint_dataset2['VENTURE_PC_GDP'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_BES_MIO_EUR

d =  joint_dataset2
x1 = joint_dataset2["GERDS_BES_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_BES_MIO_EUR
q1 = joint_dataset2['GERDS_BES_MIO_EUR'].quantile(.25)
q3 = joint_dataset2['GERDS_BES_MIO_EUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_GOV_EUR_HAB

d =  joint_dataset2
x1 = joint_dataset2["GERDS_GOV_EUR_HAB"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_GOV_EUR_HAB
q1 = joint_dataset2['GERDS_GOV_EUR_HAB'].quantile(.25)
q3 = joint_dataset2['GERDS_GOV_EUR_HAB'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_GOV_MIO_EUR

d =  joint_dataset2
x1 = joint_dataset2["GERDS_GOV_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_GOV_MIO_EUR
q1 = joint_dataset2['GERDS_GOV_MIO_EUR'].quantile(.25)
q3 = joint_dataset2['GERDS_GOV_MIO_EUR'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_HES_EUR_HAB 

d =  joint_dataset2
x1 = joint_dataset2["GERDS_HES_EUR_HAB"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

In [ ]:
# Quartiles of GERDS_HES_EUR_HAB
q1 = joint_dataset2['GERDS_HES_EUR_HAB'].quantile(.25)
q3 = joint_dataset2['GERDS_HES_EUR_HAB'].quantile(.75)

display(q1)
display(q3-q1) #IQR
display(q3)

In [ ]:
# Scatter plot - GERDS_HES_MIO_EUR 

d =  joint_dataset2
x1 = joint_dataset2["GERDS_HES_MIO_EUR"]
y1 = joint_dataset2["GEO"]


sns.scatterplot(data=d, x=x1, y= y1)

# Modeling and Evaluation

In this phase we want to accomplish the following:
- Variables Normalization
- Feature Selection
- Split or dataset into train and test datasets (80% - 20%)
- Create different Regression Models with the following characteristics:
    - Targets: a measure of the Value-Added in High-Tech Manufacturing, a measure of the Value-Added in Medium-High-Tech Manufacturing, a measure of the Value-Added in Knowledge Intensive Services
    - Features: a selection of features based on different strategies - Select K-Best technique, Multicollinearity and significance for our regresison models.
- Evaluate models that at least fulfill the assumption of residual independence.

## Variables Normalization

Because we want all the different metric variables to have the same base importance when creating the different models, and because they represent different magnitudes (percentages, number of companies, currency amounts), we must first normalize them, ie, apply a transformation based on each minimum and maximum - minmaxscaler. 

We decided not to normalize percentage based variables, since they already are normalized on a scale from 0 to 100%. On the other hand, we normalize all variables pertaining to amount of currency (million Euro or Euro per habitant) or quantities.

In [ ]:
joint_dataset2.columns[2:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaled_names = ['C_HTC_CP_MEUR', 'C_HTC_M_CP_MEUR',
       'KIS_CP_MEUR', 'VENTURE_MIO_EUR', 'VENTURE_NR_COMP', 
       'GERDS_BES_EUR_HAB', 'GERDS_BES_MIO_EUR', 
       'GERDS_GOV_EUR_HAB', 'GERDS_GOV_MIO_EUR',
       'GERDS_HES_EUR_HAB', 'GERDS_HES_MIO_EUR', 
       'GERDS_PNP_EUR_HAB', 'GERDS_PNP_MIO_EUR']
mm_scaler = MinMaxScaler()
df_joint_norm = joint_dataset2.copy()
df_joint_norm[scaled_names] = pd.DataFrame(mm_scaler.fit_transform(joint_dataset2[scaled_names]))

display(df_joint_norm.head())

In [ ]:
df_joint_norm.columns

## Feature Selection

Given the number of features per target variable, we decided to apply a method of reducing the number of features we will be using for our models.

We decided on using scikit learn's [^SelectKBest] method, for its simplicity and suitability for our use case. Given that we are going to be performing regressions, we use it with the [^f_regression] score function. This score function performs univariate linear regression tests, meaning it computes the correlation between each regressor and its target and derives a p-value from it. 

We perform the selection for each target variable, keeping only significant variables - p-value of 5% or less.


[^SelectKBest]: [sklearn.feature_selection.SelectKBes at scikit learn's website](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)  
[^f_regression]: [sklearn.feature_selection.f_regression at scikit learn's website](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)

In [ ]:
# Feature selection method: SelectKBest

# Score function:
# For regression: f_regression, mutual_info_regression
# For classification: chi2, f_classif, mutual_info_classif

########################################################################################
feature_cols = ['VENTURE_MIO_EUR', 'VENTURE_NR_COMP', 'VENTURE_PC_GDP', 
    'EMPLOYMENT_C_HTC_MH', 'EMPLOYMENT_KIS', 
    'GERDS_BES_EUR_HAB', 'GERDS_BES_MIO_EUR', 
    'GERDS_GOV_EUR_HAB', 'GERDS_GOV_MIO_EUR', 
    'GERDS_HES_EUR_HAB', 'GERDS_HES_MIO_EUR', 
    'GERDS_PNP_EUR_HAB', 'GERDS_PNP_MIO_EUR']
target_cols = ["C_HTC_PERC_OF_MANUF", "C_HTC_CP_MEUR", "C_HTC_M_PERC_OF_MANUF", "C_HTC_M_CP_MEUR", "KIS_PERC_OF_SERV", "KIS_CP_MEUR"]

# Import SelectKBest, f_regression (score function for regression)
from sklearn.feature_selection import SelectKBest, f_regression
# f_regression - F-value between label/feature for regression tasks.
 
interestingFeatures = {}

for trg in target_cols:
    featureSel = SelectKBest(score_func=f_regression, k=13)
    fit = featureSel.fit(joint_dataset2[feature_cols], joint_dataset2[trg])

    res = pd.concat([pd.DataFrame({trg: df_joint_norm[feature_cols].columns}), pd.DataFrame({"f-score":fit.scores_}), pd.DataFrame({"p-value":fit.pvalues_})], axis="columns")
    interestingFeatures[trg] = res[res["p-value"] <= 0.05][trg].values
    # for country in countries_main:
    #    fit = featureSel.fit(df_joint_norm[df_joint_norm["GEO"] == country][feature_cols], df_joint_norm[df_joint_norm["GEO"] == country][trg])
    #    res = pd.concat([res, pd.DataFrame({country + " f-score":fit.scores_})], axis="columns")

    display(res.sort_values("f-score", ascending=False))

display(interestingFeatures)

In [ ]:
df_joint_norm.isna().sum()

### Split data set into train (80%) & test (20%) groups:


In [ ]:
from sklearn.model_selection import train_test_split

feature_cols2 = ["GEO", "TIME_PERIOD"]
feature_cols2.extend(feature_cols)
x_train, x_test, y_train, y_test = train_test_split(df_joint_norm[feature_cols2], df_joint_norm[target_cols], stratify=df_joint_norm["GEO"],test_size=0.20, random_state=42)

## Regression Model - Ordinary Least Squares (OLS)

In [ ]:
# sns.mpl.rcParams['figure.figsize'] = (15.0, 6.5)
np.random.seed(42)

bpnames = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']

def linearity_test(model, y, targetvar):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    * targetvar - name of the target variable
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title(targetvar + ' Observed vs. Predicted', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title(targetvar + ' Residuals vs. Predicted', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')


def check_homoscedasticity(model):
    testres = sms.het_breuschpagan(model.resid, model.model.exog)
    display(pd.DataFrame(testres, 
                           columns=['value'],
                           index=bpnames).style.set_caption("Breusch-Pagan Lagrange Multiplier test for heteroscedasticity"))
    display("   Assumption 2 - Homoscedasticity of residuals --> {}".format(testres[1]>= 0.05))


def check_residuals_independence(model):
    #perform Breusch-Godfrey test at order p = 3
    testres= dg.acorr_breusch_godfrey(model, nlags=3)
    display(pd.DataFrame(testres, 
                           columns=['value'],
                           index=bpnames).style.set_caption("Breusch-Godfrey Lagrange Multiplier tests for residual autocorrelation"))    
    display("    Assumption 3 - Independence of residuals --> {}".format(testres[1]>= 0.05))
    

def check_residuals_normaldist(model):
    plt.subplots(1,2)
    
    plt.subplot(1,2,1)
    plt.title(model.model.endog_names + " Normal distribution of residuals")
    sns.distplot(model.resid , fit=norm)
    plt.xlabel('Residuals')    

    plt.subplot(1,2,2)
    stats.probplot(model.resid, plot=plt)
    
    # Jarque-Bera - "expectation 4 - normal distribution of residuals"
    testres = sms.jarque_bera(model.resid)
    display(pd.DataFrame(testres, columns=['value'], index=['Jarque-Bera', 'Chi^2 two-tail prob.', 'Skew', 'Kurtosis']).style.set_caption("Jarque-Bera test of normality"))    
    display("    Assumption 4 -normal distribution of residuals --> {}".format(testres[1]>= 0.05))


def check_VIF(X): 
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    display(pd.DataFrame({'vif': vif[1:]}, index=X.columns[1:]).T.style.set_caption("VIF (variance inflation factor) multicollinearity test"))
    
    
def check_assumptions(trg_name, features_list, x_data, y_data): # editei JM
    display("============ TARGET: " + trg_name)
    X_constant = sm.add_constant(x_data[features_list[trg_name]])
    lin_reg = sm.OLS(y_data[trg_name],X_constant).fit()
    display(lin_reg.summary())
    display("Assumption 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
    check_homoscedasticity(lin_reg)
    check_residuals_independence(lin_reg)
    check_residuals_normaldist(lin_reg)
    check_VIF(X_constant)
    linearity_test(lin_reg, y_data[trg_name], trg_name)  


def cross_validation_linear(trg_name, features_list, x_data, y_data, k_fold=10, score='r2'): # as standard, compute 10-fold cross validation
    scores = cross_val_score(LinearRegression(), x_data[features_list[trg_name]], y_data[trg_name],cv=k_fold, scoring=score)
    print("Model {} Score: ".format(score), round(scores.mean(), 2))
    scores = cross_val_score(LinearRegression(), x_data[features_list[trg_name]], y_data[trg_name],cv=k_fold, scoring="neg_mean_absolute_error")
    print("Model MAE Score: ", abs(round(scores.mean(), 2)))
    scores = cross_val_score(LinearRegression(), x_data[features_list[trg_name]], y_data[trg_name],cv=k_fold, scoring="neg_mean_squared_error")
    print("Model MSE Score: ", abs(round(scores.mean(), 2)))


def cross_validation_poly(trg_name, features_list, x_data, y_data, k_fold=10, score='r2'): # as standard, compute 10-fold cross validation
    poly = PolynomialFeatures(degree = 2) 
    x_poly = poly.fit_transform(x_data[features_list[trg_name]]) 
    scores = cross_val_score(LinearRegression(), x_poly, y_data[trg_name],cv=k_fold, scoring=score)
    print("Model {} Score: ".format(score), round(scores.mean(), 2))
    scores = cross_val_score(LinearRegression(), x_poly, y_data[trg_name],cv=k_fold, scoring="neg_mean_absolute_error")
    print("Model MAE Score: ", abs(round(scores.mean(), 2)))
    scores = cross_val_score(LinearRegression(), x_poly, y_data[trg_name],cv=k_fold, scoring="neg_mean_squared_error")
    print("Model MSE Score: ", abs(round(scores.mean(), 2)))

def cross_validation_log(trg_name, features_list, x_data, y_data, k_fold=10, score='r2'): # as standard, compute 10-fold cross validation
    scores = cross_val_score(LinearRegression(), x_data[features_list[trg_name]], np.log(y_data[trg_name]),cv=k_fold, scoring=score)
    print("Model {} Score: ".format(score), round(scores.mean(), 2))
    scores = cross_val_score(LinearRegression(), x_data[features_list[trg_name]], np.log(y_data[trg_name]),cv=k_fold, scoring="neg_mean_absolute_error")
    print("Model MAE Score: ", abs(round(scores.mean(), 2)))
    scores = cross_val_score(LinearRegression(), x_data[features_list[trg_name]], np.log(y_data[trg_name]),cv=k_fold, scoring="neg_mean_squared_error")
    print("Model MSE Score: ", abs(round(scores.mean(), 2)))

def model_evaluation(trg_name, features_list, degree=1, log=False): # as standard, compute 10-fold cross validation
    poly = PolynomialFeatures(degree = degree) 
    x_poly_train = poly.fit_transform(x_train[features_list[trg_name]]) 
    x_poly_test = poly.fit_transform(x_test[features_list[trg_name]]) 

    if log:
        new_y_train = np.log(y_train[trg_name][y_train[trg_name]>0])
        new_y_test = np.log(y_test[trg_name][y_test[trg_name]>0])
        x_poly_train = x_poly_train[y_train[trg_name]>0]
        x_poly_test = x_poly_test[y_test[trg_name]>0]
    else:
        new_y_train = y_train[trg_name]
        new_y_test = y_test[trg_name]

    # create and train model
    model = LinearRegression()
    model.fit(x_poly_train, new_y_train)
    # predict
    y_pred = model.predict(x_poly_test)
    mse = np.mean((y_pred - new_y_test)**2)
    mae = np.mean(abs((y_pred - new_y_test)))
    rsquare= model.score(x_poly_test, new_y_test)
    display("MSE = {}".format(mse))
    display("MAE = {}".format(mae))
    display("R² = {}".format(rsquare))
    
    if log:
        y_pred=np.exp(y_pred)
        new_y_test=np.exp(new_y_test)


    resids = np.abs(new_y_test-y_pred)
    resids.name = resids.name+"_ABS_ERROR"
    display("=== TOP 5 prediction errors:", x_test.loc[resids.sort_values().tail().index][["GEO", "TIME_PERIOD"]].join(resids.sort_values().tail(), how="inner"))

    fig, ax = plt.subplots(1,2)
    sns.regplot(x=y_pred, y=new_y_test, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title(trg_name + ' Observed vs. Predicted', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    plt.plot(range(new_y_test.size), new_y_test, color="red")
    plt.plot(y_pred, color="blue")
    plt.xlabel("Observation")
    plt.ylabel(trg_name)
    plt.legend(["observed", "predicted"], loc="upper right")
    
targets = ["C_HTC_PERC_OF_MANUF", "C_HTC_CP_MEUR", "C_HTC_M_PERC_OF_MANUF", "C_HTC_M_CP_MEUR", "KIS_PERC_OF_SERV", "KIS_CP_MEUR"]


In [ ]:
# Lasso specific:
class model_wrapper:
    def __init__(self, model, resids, y, target_name):
        self.model = model
        self.model.exog = y
        self.model.endog_names = target_name
        self.resid = resids
    def predict(self):
        return self.model.exog

def AIC_BIC(X, y, residuals):
    n = y.size
    k = X.columns.size + 1 # add intercept
    # print("n={}; k={}".format(n, k))

    #log likelihood
    ll = -(n * 1/2) * (1 + np.log(2 * np.pi)) - (n / 2) * np.log(residuals.dot(residuals) / n)

    # AIC + BIC
    AIC = (-2 * ll) + (2 * k)
    BIC = (-2 * ll) + (k * np.log(n))

    return AIC, BIC

def lasso_linearity_test(y, fitted_vals, residuals, targetvar):
    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title(targetvar + ' Observed vs. Predicted', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=residuals, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title(targetvar + ' Residuals vs. Predicted', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')

clf = linear_model.Lasso(alpha=0.1)

def lasso_regression_current_target():
    display("============ TARGET: " + target)

    my_y = y_train.copy()
    my_y=my_y.drop(my_y.columns.difference([target]), 1)

    lin_reg = clf.fit(x_train[feature_cols], my_y)
    display(pd.DataFrame([np.insert(lin_reg.coef_,0,lin_reg.intercept_)], columns=["intercept"]+feature_cols))
    display("R² = {}".format(clf.score(x_train[feature_cols], my_y)))

    y_pred = lin_reg.predict(x_train[feature_cols])

    resids = y_train[target]-y_pred
    AICBIC = AIC_BIC(x_train[feature_cols], my_y, resids)
    display(pd.DataFrame({"AIC": [AICBIC[0]], "BIC": [AICBIC[1]]}))
    lasso_linearity_test(my_y, y_pred, resids, target)
    display("    Expectation 1 - mean of residuals is zero: {} --> {}".format(resids.mean(), round(resids.mean(),1) == 0.0))
    check_homoscedasticity(model_wrapper(lin_reg, resids, x_train[feature_cols], target))
    check_residuals_independence(model_wrapper(lin_reg, resids, x_train[feature_cols], target))
    check_residuals_normaldist(model_wrapper(lin_reg, resids, x_train[feature_cols], target))
    

### Stepwise feature elimination

Given the still high number of features per target variable and the fact that some of them are correlated and create problems of multicollinearity for our models, we decided on eliminating features. For that, we decided on performing stepwise elimination using two methods:
- Eliminate non-significant features by building a Multivariate Linear Regression model and - at each step - eliminate the most non-significant variable (the variable with the highest p-value, above 0.05).
- Eliminate variables that are highly correlated to other variables in the model and are considered to create multicollinearity problems. At each step we check the VIF (variance inflation factor) of all the variables still considered for the target variable, we eliminate the variable with the highest vif, as long as it is above five. This way we make sure from highly correlated variables at least one significant one is kept in the model.


In [ ]:
# stepwise feature elimination using Multivariate linear regression p-values
for trg_name in target_cols:
    while True:
        X_constant = sm.add_constant(x_train[interestingFeatures[trg_name]])
        lin_reg = sm.OLS(y_train[trg_name], X_constant).fit()
        pvals = round(lin_reg.pvalues[1:],3)
        if pvals.size == 0 or pvals.sort_values(ascending=False)[0] <= 0.05:
            break
        else:
            interestingFeatures[trg_name] = interestingFeatures[trg_name][interestingFeatures[trg_name] != pvals.sort_values(ascending=False).index[0]]
            # display(pvals.sort_values(ascending=False)[0], pvals.sort_values(ascending=False).index[0])
 
# stepwise feature elimination based on VIF
for trg_name in target_cols:
    while True:
        X_constant = sm.add_constant(x_train[interestingFeatures[trg_name]])
        #display(interestingFeatures[trg_name], X_constant.shape[1])
        vif =  [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
        vif = vif[1:]
        vif = pd.Series(vif)
        vif = vif[vif > 5]
        #display(vif)
        #display(vif.sort_values(ascending=False))
        #display(vif.sort_values(ascending=False).index[0])
        if vif.size == 0:
            break
        else:
            #display(vif)
            #display(interestingFeatures[trg_name], vif.sort_values(ascending=False).index[0])        
            interestingFeatures[trg_name] = np.delete(interestingFeatures[trg_name], vif.sort_values(ascending=False).index[0])
 
display(interestingFeatures)

## Modeling and validation target variable 1: C_HTC_PERC_OF_MANUF 

 High Tech industry - added value as percentage of manufacturing

### Model 1
With this model we are picking only the variables selected with the SelectKBest method before and also only the signigficant ones that don't cause multicollinearity problems.


In [ ]:
target = "C_HTC_PERC_OF_MANUF"

sns.pairplot(df_joint_norm[[target] + interestingFeatures[target].tolist()])
check_assumptions(target, interestingFeatures, x_train, y_train)



Recap of Model 1 for target `C_HTC_PERC_OF_MANUF` - High Tech industry - added value as percentage of manufacturing:

- Adj. R-squared:	0.323
  
- AIC:	922.9
  
- Assumptions not met: 

  2 - Homoscedasticity of residuals; 
  4 – Normal distribution of residuals; 

Strategies to apply to next model:  
    - We will try a polynomial model to improve our model.     

### Target Variable 1 Cross-Validation With Model 1 and 10-fold method:

In [ ]:
cross_validation_linear(target, interestingFeatures, x_train, y_train, k_fold=10)

In [ ]:
model_evaluation("C_HTC_PERC_OF_MANUF", interestingFeatures)

### Model 2 - Try the polynomial feature transformation



In [ ]:
from sklearn.preprocessing import PolynomialFeatures 
poly = PolynomialFeatures(degree = 2) 
target = "C_HTC_PERC_OF_MANUF"
display("============ TARGET: " + target)

# C_HTC_PERC_OF_MANUF Using polinomial features
X_poly = poly.fit_transform(x_train[interestingFeatures[target]])  
poly.fit(X_poly, y_train[target]) 
X_poly = sm.add_constant(X_poly)
results = sm.OLS(y_train[target], X_poly).fit()
display(results.summary())
display("    Expectation 1 - mean of residuals is zero: {} --> {}".format(results.resid.mean(), round(results.resid.mean(),1) == 0.0))
check_homoscedasticity(results)
check_residuals_independence(results)
check_residuals_normaldist(results)
vif = [variance_inflation_factor(X_poly, i) for i in range(X_poly.shape[1])]
display(pd.DataFrame({'vif': vif[1:]}).T.style.set_caption("VIF (variance inflation factor) multicollinearity test"))
linearity_test(results, y_train[target], target)
#display(poly.get_feature_names(x_train[interestingFeatures[target]].columns))

#### Model 2 round-up:  

Recap of target `C_HTC_PERC_OF_MANUF` - High Tech industry - added value as percentage of manufacturing:

- Adj. R-squared:	0.389 (better then 0.323 in model 1)
  
- AIC:	915.5 (slightly worst then the model 1's 922.9)
  
- Assumptions not met stay the same: 

  2 - Homoscedasticity of residuals; 
  4 – Normal distribution of residuals; 
  
  
Strategies to apply to next model:  
    - We will try a non linear transformation.

Testing Cross-validation of the model:

In [ ]:
cross_validation_poly(target, interestingFeatures, x_train, y_train, k_fold=10)

In [ ]:
model_evaluation("C_HTC_PERC_OF_MANUF", interestingFeatures, degree=2)

### Model 3 - non-linear transformation

Since we will use log of the target variable, let's check any existing  zeros.

In [ ]:
# Check zeros in our target variable
display(joint_dataset2[ ["TIME_PERIOD", "GEO",target] + interestingFeatures[target].tolist() ][joint_dataset2[target]<=0])

Seems like LU (Luxemburg) has all zeros for these target variables, so we will exclude it before moving on with the new model.

Zeros in our current training set:

In [ ]:
# Check zeros in our target variable
display(x_train[ ["TIME_PERIOD", "GEO"] + interestingFeatures[target].tolist() ][y_train[target]<=0])

In [ ]:
# remove zeros (all from LU, as seen above)
df = pd.merge(y_train[y_train[target]!=0], x_train[interestingFeatures[target]][y_train[target]!=0], left_index=True, right_index=True)
formula="np.log({})".format(target) + " ~ " + " + ".join(interestingFeatures[target])
print("formula: ", formula)
lin_reg = sm.OLS.from_formula(formula, df).fit()
display(lin_reg.summary())
display("    Expectation 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
check_homoscedasticity(lin_reg)
check_residuals_independence(lin_reg)
check_residuals_normaldist(lin_reg)
check_VIF(df[interestingFeatures[target]])
linearity_test(lin_reg, df[target], target)    


#### Model 3 round-up:  

Recap of target `C_HTC_PERC_OF_MANUF` - High Tech industry - added value as percentage of manufacturing:

- Adj. R-squared:	0.416  (better then 0.389 in model 2 and  0.323 in model 1)
  
- AIC:	184.1 (much worse then 922.9 in model 1 and 915.5 in model 2)
  
- Assumptions not met stay the same: 

  2 - Homoscedasticity of residuals; 
  4 – Normal distribution of residuals; 
  
Model 2 is the best up to now.

Strategies to apply to next model:  
    - Lasso regresison

In [ ]:
cross_validation_log(target, interestingFeatures, x_train[interestingFeatures[target]][y_train[target]!=0], y_train[y_train[target]!=0], k_fold=10)

In [ ]:
model_evaluation("C_HTC_PERC_OF_MANUF", interestingFeatures, log=True)

### Model 4 - Lasso regression

In [ ]:
lasso_regression_current_target()

#### Model 4 round-up:  

Recap of target `C_HTC_PERC_OF_MANUF` - High Tech industry - added value as percentage of manufacturing:

- Adj. R-squared:	0.359  (worst then model 3's 0.416 and 0.389 in model 2)
  
- AIC:	not comparable (much worse then 847.8 in model 1 and 843.0 in model 2)
  
- Assumptions not met stay the same: 

  2 - Homoscedasticity of residuals; 
  3 - Independence of residuals;
  4 – Normal distribution of residuals; 
  
Model 2 is the best up to now.

## Modeling and validation target variable 2: C_HTC_CP_MEUR

High tech added value in million Euro

### Model 1
With this model we are picking only the variables selected with the SelectKBest method before.


In [ ]:
target = "C_HTC_CP_MEUR"

sns.pairplot(df_joint_norm[[target] + interestingFeatures[target].tolist()])
check_assumptions(target, interestingFeatures, x_train, y_train)

Recap target `C_HTC_CP_MEUR` - High tech added value in million Euro:

- Adj. R-squared:	0.888
  
- AIC:	-318.7 
  
- Assumptions not met: 
  
  2 - Homoscedasticity of residuals; 
  4 – Normal distribution of residuals; 

  Strategies for next model :
    - Apply polynomial feature transformation
   

In [ ]:
cross_validation_linear(target, interestingFeatures, x_train, y_train, k_fold=10)

In [ ]:
model_evaluation("C_HTC_CP_MEUR", interestingFeatures)

### Model 2 - Try the polynomial feature transformation



In [ ]:
from sklearn.preprocessing import PolynomialFeatures 
poly = PolynomialFeatures(degree = 2) 

display("============ TARGET: " + target)

# C_HTC_PERC_OF_MANUF Using polinomial features
X_poly = poly.fit_transform(x_train[interestingFeatures[target]])  
poly.fit(X_poly, y_train[target]) 
X_poly = sm.add_constant(X_poly)
results = sm.OLS(y_train[target], X_poly).fit()
display(results.summary())
display("    Expectation 1 - mean of residuals is zero: {} --> {}".format(results.resid.mean(), round(results.resid.mean(),1) == 0.0))
check_homoscedasticity(results)
check_residuals_independence(results)
check_residuals_normaldist(results)
vif = [variance_inflation_factor(X_poly, i) for i in range(X_poly.shape[1])]
display(pd.DataFrame({'vif': vif[1:]}).T.style.set_caption("VIF (variance inflation factor) multicollinearity test"))
linearity_test(results, y_train[target], target)


Recap target `C_HTC_CP_MEUR` - High tech added value in million Euro:

- Adj. R-squared:	0.970 (better then 0.888 in Model 1 but maybe pointing at overfitting?)
  
- AIC:	-473.4 (much worse then -318.7)
  
- Assumptions not met: 
  
  2 - Homoscedasticity of residuals; 
  4 – Normal distribution of residuals; 

  
Strategies to apply to next model:  
    - We will try a non linear transformation.

In [ ]:
cross_validation_poly(target, interestingFeatures, x_train, y_train, k_fold=10)

In [ ]:
model_evaluation("C_HTC_CP_MEUR", interestingFeatures, degree=2)

### Model 3 - non-linear transformation

Since we will use log of the target variable, let's check any existing  zeros.

In [ ]:
# Check zeros in our target variable
display(joint_dataset2[ ["TIME_PERIOD", "GEO", target] + interestingFeatures[target].tolist() ][joint_dataset2[target]<=0])

Seems like LU (Luxemburg) has all zeros for these target variables, so we will exclude it before moving on with the new model.

In [ ]:
# remove zeros (all from LU, as seen above)
df = pd.merge(y_train[y_train[target]!=0], x_train[interestingFeatures[target]][y_train[target]!=0], left_index=True, right_index=True)
formula="np.log({})".format(target) + " ~ " + " + ".join(interestingFeatures[target])
print("formula: ", formula)
lin_reg = sm.OLS.from_formula(formula, df).fit()
display(lin_reg.summary())
display("    Expectation 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
check_homoscedasticity(lin_reg)
check_residuals_independence(lin_reg)
check_residuals_normaldist(lin_reg)
check_VIF(df[interestingFeatures[target]])
linearity_test(lin_reg, df[target], target)    


#### Model 3 round-up:  

Target `C_HTC_CP_MEUR` - High tech added value in million Euro

- Features: 
  - "EMPLOYMENT_C_HTC_MH" - Employment in High Tech and Medium High Tech industries
  - "EMPLOYMENT_KIS" - Employment in Knowledge Intensive Services;
  - "GERDS_BES_EUR_HAB" - Gross private expenditure in Research and Development, in Euro per habitant;
  - "GERDS_BES_MIO_EUR" - Gross private expenditure in Research and Development, in million Euro.
  - "GERDS_GOV_EUR_HAB" - Gross government expenditure in Research and Development, in Euro per habitant;
 
- Adj. R-squared:	0.641	(much worse then the Model 2 0.970)
  
- AIC:	287.6 (much better then Model 2's -473.4)
  
- Assumptions not met are reduced to: 
  
  2 - Homoscedasticity of residuals; 

We were able to improve the normal distribution of residuals.
 
Strategies to apply to next model, in order to try to fix the assumptions that are not met:  
    - Try a Lasso regression.  
  

In [ ]:
cross_validation_log(target, interestingFeatures, x_train[interestingFeatures[target]][y_train[target]!=0], y_train[y_train[target]!=0], k_fold=10)

In [ ]:
model_evaluation("C_HTC_CP_MEUR", interestingFeatures, log=True)

### Model 4 - Lasso regression

In [ ]:
lasso_regression_current_target()

Model 4 is not a good model.

## Modeling and validation target variable 3: C_HTC_M_PERC_OF_MANUF


### Model 1:


In [ ]:
x_trg3_mod1 = x_train.copy()
y_trg3_mod1 = y_train.copy()
check_assumptions('C_HTC_M_PERC_OF_MANUF', interestingFeatures, x_trg3_mod1, y_trg3_mod1)

#### Model 1 round-up:

 - Target: C_HTC_M_PERC_OF_MANUF

 - Features: VENTURE_PC_GDP,	EMPLOYMENT_C_HTC_MH,	GERDS_HES_EUR_HAB,	GERDS_HES_MIO_EUR
 
 - Adj. R-Squared: 0.681
 
 - AIC: 836.9

 - Assumptions not met: 4 – Normal distribution of residuals

### Target Variable 3 Cross-Validation With Model 1 and 10-fold method:

In [ ]:
cross_validation_linear('C_HTC_M_PERC_OF_MANUF', interestingFeatures, x_trg3_mod1, y_trg3_mod1, k_fold=10)

In [ ]:
model_evaluation("C_HTC_M_PERC_OF_MANUF", interestingFeatures)

### Model 2:

In [ ]:
x = x_trg3_mod1[interestingFeatures['C_HTC_M_PERC_OF_MANUF']].copy()
y = y_trg3_mod1['C_HTC_M_PERC_OF_MANUF'].copy()

# features transformation in order to try to grasp the non-linearity (2 degree polynomial):
poly = PolynomialFeatures(degree = 2) 
x_poly = poly.fit_transform(x) 
poly.fit(x_poly, y) 
x_poly = sm.add_constant(x_poly)


# check_assumptions:
display("============ TARGET: 'C_HTC_M_PERC_OF_MANUF' ")
X_constant = x_poly
lin_reg = sm.OLS(y,X_constant).fit()
display(lin_reg.summary())
display("Expectation 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
check_homoscedasticity(lin_reg)
check_residuals_independence(lin_reg)
check_residuals_normaldist(lin_reg)
linearity_test(lin_reg, y, 'C_HTC_M_PERC_OF_MANUF') 
check_VIF(pd.DataFrame(X_constant))

#### Model 2 round-up:

 - Target: C_HTC_M_PERC_OF_MANUF

 - Features: VENTURE_PC_GDP, EMPLOYMENT_C_HTC_MH, GERDS_HES_EUR_HAB, GERDS_HES_MIO_EUR
 
 - Adj. R-Squared: 0.751
 
 - AIC: 814.1

 - Assumptions not met: 2 - Homoscedasticity of residuals; 4 – Normal distribution of residuals

 

In [ ]:
cross_validation_poly('C_HTC_M_PERC_OF_MANUF', interestingFeatures, x_trg3_mod1, y_trg3_mod1, k_fold=10)

In [ ]:
model_evaluation("C_HTC_M_PERC_OF_MANUF", interestingFeatures, degree=2)

### Model 3: Lasso

In [ ]:
target = 'C_HTC_M_PERC_OF_MANUF'
lasso_regression_current_target()

## Modeling and validation target variable 4: C_HTC_M_CP_MEUR


Let's create a regression model with C_HTC_M_CP_MEUR as the target variable we want to predict.

### Model 1:

In [ ]:
x_trg4_mod1 = x_train.copy()
y_trg4_mod1 = y_train.copy()
check_assumptions('C_HTC_M_CP_MEUR', interestingFeatures, x_trg4_mod1, y_trg4_mod1)

#### Model 1 round-up:  
  
- Target: C_HTC_M_CP_MEUR  
  
- Features: VENTURE_MIO_EUR,	EMPLOYMENT_C_HTC_MH,	GERDS_BES_EUR_HAB,	GERDS_BES_MIO_EUR,	GERDS_GOV_EUR_HAB,	GERDS_PNP_MIO_EUR 
  
- Adj. R-Squared: 0.949 
  
- AIC: -449.5
  
- Assumptions not met: 2 - Homoscedasticity of residuals; 3 - Independence of residuals; 4 – Normal distribution of residuals; 5 – Variables have no coliniarity
   

### Target Variable 4 Cross-Validation With Model 1 and 10-fold method:

In [ ]:
cross_validation_linear('C_HTC_M_CP_MEUR', interestingFeatures, x_trg4_mod1, y_trg4_mod1, k_fold=10, score='r2')

In [ ]:
model_evaluation("C_HTC_M_CP_MEUR", interestingFeatures)

### Model 2:

In [ ]:
x = x_trg4_mod1[interestingFeatures['C_HTC_M_CP_MEUR']].copy()
y = y_trg4_mod1['C_HTC_M_CP_MEUR'].copy()

# features transformation in order to try to grasp the non-linearity (2 degree polynomial):
poly = PolynomialFeatures(degree = 2) 
x_poly = poly.fit_transform(x) 
poly.fit(x_poly, y) 
x_poly = sm.add_constant(x_poly)

# check_assumptions('C_HTC_M_PERC_OF_MANUF', interestingFeatures, x_poly, y)

# check_assumptions:
display("============ TARGET: 'C_HTC_M_CP_MEUR' ")
X_constant = x_poly
lin_reg = sm.OLS(y,X_constant).fit()
display(lin_reg.summary())
display("Expectation 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
check_homoscedasticity(lin_reg)
check_residuals_independence(lin_reg)
check_residuals_normaldist(lin_reg)
linearity_test(lin_reg, y, 'C_HTC_M_CP_MEUR') 
check_VIF(pd.DataFrame(X_constant))

#### Model 2 round-up:  
  
- Target: C_HTC_M_CP_MEUR  
  
- Features: VENTURE_MIO_EUR, EMPLOYMENT_C_HTC_MH, GERDS_BES_EUR_HAB, GERDS_BES_MIO_EUR, GERDS_GOV_EUR_HAB, GERDS_PNP_MIO_EUR
  
- Adj. R-Squared: 0.995
  
- AIC: -732.9
  
- Assumptions not met: 2 - Homoscedasticity of residuals; 4 – Normal distribution of residuals
   


In [ ]:
cross_validation_poly('C_HTC_M_CP_MEUR', interestingFeatures, x_trg4_mod1, y_trg4_mod1, k_fold=10, score='r2')

In [ ]:
model_evaluation("C_HTC_M_CP_MEUR", interestingFeatures, degree=2)

Way too many independent variables; most likely to overfit.

### Model 3: Lasso

In [ ]:
target = 'C_HTC_M_CP_MEUR'
lasso_regression_current_target()

## Modeling and validation target variable 5: KIS_PERC_OF_SERV

In [ ]:
x_trg5_mod1 = x_train.copy()
y_trg5_mod1 = y_train.copy()
check_assumptions('KIS_PERC_OF_SERV', interestingFeatures, x_trg5_mod1, y_trg5_mod1)

#### Model 1 round-up:  
  
- Target: KIS_PERC_OF_SERV  
  
- Features: VENTURE_PC_GDP,	EMPLOYMENT_KIS,	GERDS_HES_EUR_HAB
  
- Adj. R-Squared: 0.639
  
- AIC: 771.8 
  
- Assumptions not met: 2 - Homoscedasticity of residuals; 4 -normal distribution of residuals
   

In [ ]:
cross_validation_linear('KIS_PERC_OF_SERV', interestingFeatures, x_trg5_mod1, y_trg5_mod1)

In [ ]:
model_evaluation("KIS_PERC_OF_SERV", interestingFeatures)

### Model 2:

In [ ]:
x = x_trg5_mod1[interestingFeatures['KIS_PERC_OF_SERV']].copy()
y = y_trg5_mod1['KIS_PERC_OF_SERV'].copy()

# features transformation in order to try to grasp the non-linearity (2 degree polynomial):
poly = PolynomialFeatures(degree = 2) 
x_poly = poly.fit_transform(x) 
poly.fit(x_poly, y) 
x_poly = sm.add_constant(x_poly)

# check_assumptions('C_HTC_M_PERC_OF_MANUF', interestingFeatures, x_poly, y)

# check_assumptions:
display("============ TARGET: 'KIS_PERC_OF_SERV' ")
X_constant = x_poly
lin_reg = sm.OLS(y,X_constant).fit()
display(lin_reg.summary())
display("Expectation 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
check_homoscedasticity(lin_reg)
check_residuals_independence(lin_reg)
check_residuals_normaldist(lin_reg)
linearity_test(lin_reg, y, 'KIS_PERC_OF_SERV') 
check_VIF(pd.DataFrame(X_constant))

#### Model 2 round-up:  
  
- Target: KIS_PERC_OF_SERV  
  
- Features: VENTURE_PC_GDP,	EMPLOYMENT_KIS,	GERDS_HES_EUR_HAB
  
- Adj. R-Squared: 0.738
  
- AIC: 736.3
  
- Assumptions not met: 2 - Homoscedasticity of residuals; 4 -normal distribution of residuals

In [ ]:
cross_validation_poly('KIS_PERC_OF_SERV', interestingFeatures, x_trg5_mod1, y_trg5_mod1)

In [ ]:
model_evaluation("KIS_PERC_OF_SERV", interestingFeatures, degree=2)

### Model 3: Lasso

In [ ]:
target = 'KIS_PERC_OF_SERV'
lasso_regression_current_target()

## Modeling and validation target variable 6: KIS_CP_MEUR

### Model 1

In [ ]:
x_trg6_mod1 = x_train.copy()
y_trg6_mod1 = y_train.copy()
check_assumptions('KIS_CP_MEUR', interestingFeatures, x_trg6_mod1, y_trg6_mod1)

#### Model 1 round-up:  
  
- Target: KIS_CP_MEUR  
  
- Features: 	VENTURE_MIO_EUR,	EMPLOYMENT_C_HTC_MH,	EMPLOYMENT_KIS,	GERDS_GOV_MIO_EUR,	GERDS_PNP_EUR_HAB,	GERDS_PNP_MIO_EUR
  
- Adj. R-Squared: 0.958 
  
- AIC: -369.0
  
- Assumptions not met: 2 - Homoscedasticity of residuals; 4 – Normal distribution of residuals


In [ ]:
cross_validation_linear('KIS_CP_MEUR', interestingFeatures, x_trg6_mod1, y_trg6_mod1)

In [ ]:
model_evaluation("KIS_CP_MEUR", interestingFeatures)

### Model 2:

In [ ]:
x = x_trg6_mod1[interestingFeatures['KIS_CP_MEUR']].copy()
y = y_trg6_mod1['KIS_CP_MEUR'].copy()

# features transformation in order to try to grasp the non-linearity (2 degree polynomial):
poly = PolynomialFeatures(degree = 2) 
x_poly = poly.fit_transform(x) 
poly.fit(x_poly, y) 
x_poly = sm.add_constant(x_poly)

# check_assumptions('C_HTC_M_PERC_OF_MANUF', interestingFeatures, x_poly, y)

# check_assumptions:
display("============ TARGET: 'KIS_CP_MEUR' ")
X_constant = x_poly
lin_reg = sm.OLS(y,X_constant).fit()
display(lin_reg.summary())
display("Expectation 1 - mean of residuals is zero: {} --> {}".format(lin_reg.resid.mean(), round(lin_reg.resid.mean(),1) == 0.0))
check_homoscedasticity(lin_reg)
check_residuals_independence(lin_reg)
check_residuals_normaldist(lin_reg)
linearity_test(lin_reg, y, 'KIS_CP_MEUR') 
check_VIF(pd.DataFrame(X_constant))

#### Model 2 round-up:  
  
- Target: KIS_CP_MEUR  
  
- Features: 'EMPLOYMENT_C_HTC_MH', 'GERDS_GOV_EUR_HAB', 'GERDS_PNP_EUR_HAB'
  
- Adj. R-Squared: 0.996
  
- AIC: -644.4
  
- Assumptions not met: 2 - Homoscedasticity of residuals; 4 -normal distribution of residuals


In [ ]:
cross_validation_poly('KIS_CP_MEUR', interestingFeatures, x_trg6_mod1, y_trg6_mod1)

In [ ]:
model_evaluation("KIS_CP_MEUR", interestingFeatures, degree=2)

### Model 3: Lasso

In [ ]:
target = 'KIS_CP_MEUR'
lasso_regression_current_target()

In [ ]:
interestingFeatures

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3698198a-96cb-4caf-9444-9c67ff73b730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>